In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_ver2.csv', 'test_ver2.csv', 'sample_submission.csv']


In [2]:
import xgboost as xgb

In [2]:
train = pd.read_csv('../input/train_ver2.csv')
test = pd.read_csv('../input/test_ver2.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test.shape

(929615, 24)

In [3]:
#제품이름
prods = train.columns[24:].tolist()

In [7]:
train[prods].isnull().sum()

ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
ind_reca_fin_ult1        0
ind_tjcr_fin_ult1        0
ind_valo_fin_ult1        0
ind_viv_fin_ult1         0
ind_nomina_ult1      16063
ind_nom_pens_ult1    16063
ind_recibo_ult1          0
dtype: int64

In [4]:
#제품 변수 결측치 0으로
train[prods] = train[prods].fillna(0.0).astype(np.int8)

In [5]:
#24개 제품 중 하나도 보유하지 않은 고객데이터 제거
no_product = train[prods].sum(axis=1) == 0
train = train[~no_product]

In [6]:
for col in prods :
    test[col] = 0 #일단 test 데이터에 target을 0으로 initialize
df = pd.concat([train,test], axis = 0)

In [7]:
del train
del test

In [14]:
df.shape

(12020685, 48)

In [8]:
#학습에 필요한 변수를 담는 list
features = []

### 범주형 변수들

In [9]:
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols :
    df[col], _ = df[col].factorize(na_sentinel =-99) #범주형 변수들의 factor값들을 숫자로
    
features+=categorical_cols

In [20]:
df[col].head()

0    0
1    1
2    1
3    1
4    1
Name: segmento, dtype: int64

### numeric variables

In [16]:
#NA값들 -99로 바꾸고 int 타입으로 변환
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True) #중간에 p라는 값이 있음
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [17]:
# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']